# Convert USGS rupture json files to OQ rupture xml files

##  Configuration

In [ ]:
# Standard dependencies
import os, io, sys, glob
import json
import numpy as np

# OQ baselib, hazardlib dependencies
from openquake.baselib.node import node_to_xml, Node, floatformat
from openquake.hazardlib.source.rupture import ParametricProbabilisticRupture
from openquake.hazardlib.geo.surface import PlanarSurface
from openquake.hazardlib.source.multi_fault import MultiFaultSource
from openquake.hazardlib.geo.point import Point
from openquake.hazardlib.tom import PoissonTOM

## User input

In [ ]:
# Event name
event = '20150425_M7.8_Gorkha'

# Rake angle - USGS rupture json specifies 0 for all rake angles; need to produce from the literature
rake = 999 # must be between -180 to 180
rake_source = 'Unknown'

## Read USGS rupture json

In [ ]:
# Folders and directories
file_name = "rupture_USGS.json"
json_path = glob.glob(os.path.join('..', '*', event, 
                                   'Rupture', file_name))[0]
print('Processing file:', json_path)
output_path = json_path.replace(file_name, "earthquake_rupture_model_USGS.xml")

with open(json_path) as f:
    rj = json.load(f)

# Preview
rj

## Convert to OQ rupture xml

In [ ]:
# Utility function - create OQ ruptures
def make_rup(rup_json, rake):
    
    # Extract coordinates from rupture json
    ftype = rup_json['features'][0]['geometry']['type']

    # Set rake to a reference value
    rup_json['metadata']['rake'] = rake

    if ftype == 'Point':
        raise ValueError('This notebook does not support ruptures with Point geometries.')
    
    elif ftype == 'MultiPolygon':
         # Get coordinates
         coords = np.array(rup_json['features'][0]['geometry']['coordinates'][0])
         
         # Define rupture type and coordinates
         if len(coords) == 1:
            num_points = len(coords[0]) - (len(coords[0]) % 2)

            # Single plane
            if num_points == 4:
                mode = "PlanarSurface"
                rup_coordinates = coords[0][:num_points]
            else:
                mode = "ComplexSurface"
                raise ValueError('This notebook does not support ruptures with complex geometries yet')

         else:
           mode = "MultiSurface"
           # To investigate how to handle MultiSurface
           rup_coordinates = np.array(rup_json['features'][0]
                                        ['geometry']['coordinates'][0])
        #    raise ValueError('This notebook does not support multi-plane ruptures yet')

    print(f'Coordinates: {rup_coordinates}')
    
    # Hypocenter location
    hyp_lon = rup_json['metadata']['lon']
    hyp_lat = rup_json['metadata']['lat']
    hyp_depth = rup_json['metadata']['depth']
    hypocenter = Point(hyp_lon, hyp_lat, hyp_depth)
    
     # Set tectonic region type
    trt = 'Active Shallow Crust' if hyp_depth < 50 else 'Subduction IntraSlab'
    
    # Set magnitude
    Mw = rup_json['metadata']['mag']
    
    # Create surface
    if mode == "PlanarSurface":
        surf = make_surface_from_pt_set(rup_coordinates)
        rupture = ParametricProbabilisticRupture(Mw, rake, trt, hypocenter, surf, 1.0,
                                          PoissonTOM(1.0))
    elif mode == "MultiSurface":
        surfaces = {}    
        for i, pt_set in enumerate(rup_coordinates):
            assert len(pt_set) == 5, f"""Multi planar with complex planes.
            More than 4 points in set: {pt_set}"""
            surfaces[i] = make_surface_from_pt_set(pt_set[:4])
            
        rup_idxs = [list(surfaces.keys())]    
        sections = list(surfaces.values())
        pmfs = [[0.0, 1.0]]
        source = MultiFaultSource("01",  "Multi-Fault rupture", 
                                  trt, rup_idxs, pmfs, [Mw], [rake])
        
        source.set_sections(sections)
        for rupture in source.iter_ruptures():
            rupture = rupture
    else:
        raise AttributeError(f'{mode} not supported')
    
    return rupture


def make_surface_from_pt_set(rup_coords):
    
    corner_pts = [Point(*cc) for cc in rup_coords]
    surf = PlanarSurface.from_corner_points(*corner_pts)
    
    return surf

In [ ]:
# Utility function - write OQ rupture

# Namespace
F64 = np.float64
NAMESPACE = 'http://openquake.org/xmlns/nrml/0.4'
NRML05 = 'http://openquake.org/xmlns/nrml/0.5'
GML_NAMESPACE = 'http://www.opengis.net/gml'
SERIALIZE_NS_MAP = {None: NAMESPACE, 'gml': GML_NAMESPACE}
PARSE_NS_MAP = {'nrml': NAMESPACE, 'gml': GML_NAMESPACE}

# Write node
def nwrite(nodes, output=sys.stdout, fmt='%.5f', gml=True, xmlns=None):
    """
    Convert nodes into a NRML file. output must be a file
    object open in write mode. If you want to perform a
    consistency check, open it in read-write mode, then it will
    be read after creation and validated.
    :params nodes: an iterable over Node objects
    :params output: a file-like object in write or read-write mode
    :param fmt: format used for writing the floats (default '%.7E')
    :param gml: add the http://www.opengis.net/gml namespace
    :param xmlns: NRML namespace like http://openquake.org/xmlns/nrml/0.4
    """
    root = Node('nrml', nodes=nodes)
    namespaces = {xmlns or NRML05: ''}
    if gml:
        namespaces[GML_NAMESPACE] = 'gml:'
    with floatformat(fmt):
        node_to_xml(root, output, namespaces)
    if hasattr(output, 'mode') and '+' in output.mode:  # read-write mode
        output.seek(0)
        read(output)  # validate the written file

# Convert rupture to file        
def rup_to_file(rup, outfile, commentstr):
    # Determine geometry
    geom = rup.surface.surface_nodes[0].tag
    name = ""
    if len(rup.surface.surface_nodes) > 1:
        name = 'multiPlanesRupture'
    elif geom == 'planarSurface':
        name = 'singlePlaneRupture'
    elif geom == 'simpleFaultGeometry':
        name = 'simpleFaultRupture'
    elif geom == 'complexFaultGeometry':
        name = 'complexFaultRupture'
    elif geom == 'griddedSurface':
        name = 'griddedRupture'
    elif geom == 'kiteSurface':
        name = 'kiteSurface'
    # Arrange node
    h = rup.hypocenter
    hp_dict = dict(lon=h.longitude, lat=h.latitude, depth=h.depth)
    geom_nodes = [Node('magnitude', {}, rup.mag),
                    Node('rake', {}, rup.rake),
                    Node('hypocenter', hp_dict)]
    geom_nodes.extend(rup.surface.surface_nodes)
    rupt_nodes = [Node(name, nodes=geom_nodes)]
    node = Node('nrml', nodes=rupt_nodes)
    # Write file
    with open(outfile, 'w') as ff:
        bs = io.BytesIO()
        nwrite(node, output=bs)
        ff.write(bs.getvalue().decode("utf-8"))
        ff.write(commentstr)


In [ ]:
# Create output
comment_str = f"""<!-- Rupture XML automatically generated from USGS ({rj['metadata']['id']}).
Reference: {rj['metadata']['reference']}.
Rake angle defined by {rake_source}. -->\n"""
rupture = make_rup(rj, rake)
node = rup_to_file(rupture, output_path, comment_str)

# Print some information
print(f"Wrote rupture to {output_path}\n")
print(f"    Magnitude: Mw{rupture.mag:.1f}")
print(f"    Hypocenter: {rupture.hypocenter}")
print(f"    Strike angle: {rupture.surface.get_strike():.1f}\N{DEGREE SIGN}")
print(f"    Dip angle: {rupture.surface.get_dip():.1f}\N{DEGREE SIGN}")
print(f"    Rake angle: {rupture.rake:.1f}\N{DEGREE SIGN}")

In [ ]:
more $output_path